In [2]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [3]:
dim = 3
mesh_file = "/home/aben-ham/Desktop/work/stage/manapy/mesh/3D/cube.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 4573
Number of Vertices: 1140
Local domain contruction ...
---------------------------------
---------------------------------
set att: BCdirichlet
set att: BCneumann
set att: BCneumannNH
set att: cell
set att: dirichletfaces
set att: face
set att: ghost
set att: gradcellx
set att: gradcelly
set att: gradcellz
set att: gradfacex
set att: gradfacey
set att: gradfacez
set att: gradhalocellx
set att: gradhalocelly
set att: gradhalocellz
set att: halo
set att: haloghost
set att: halotosend
set att: neumannNHfaces
set att: neumannfaces
set att: node
set att: psi
set att: psihalo
compile kernel_facetocell to cuda => signature=void(float32[:], float32[:], int32[:,:], int32)
compile kernel_celltoface to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:], int32[:], int32[:], int32[:])
compile kernel_define_halosend to cuda => signature=void(float32[:], float32[:], int32[:])
compile kernel_

In [4]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,
])

args = [
  ne.cell,
  ne.ghost,
  ne.halo,
  ne.gradcellx,
  ne.gradcelly,
  ne.gradcellz,
  ne.psi,
  ne.domain.faces.cellid,
  ne.domain.cells.faceid,
  ne.domain.faces.name,
  ne.domain.faces.halofid,
  ne.domain.cells.center,
  ne.domain.faces.center
]




---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [10]:
print(ne.psi)

[1. 1. 1. ... 1. 1. 1.]


In [6]:
from manapy.ast.functions3d import barthlimiter_3d as cpu_function
from manapy.cuda.manapy.ast.cuda_functions3d import get_kernel_barthlimiter_3d as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile barthlimiter_3d to cpu => signature=(float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:,:], int32[:], int32[:], float32[:,:], float32[:,:])
compile kernel_barthlimiter_3d to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:,:], int32[:], int32[:], float32[:,:], float32[:,:])


In [7]:
numba_fun(*args)
%timeit numba_fun(*args)

303 µs ± 45.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_barthlimiter_3d.<locals>.kernel_barthlimiter_3d at 0x7104a004bc10>) is called
1 => VarClass<float32, (4573,)>
2 => VarClass<float32, (9874,)>
3 => VarClass<float32, (0,)>
4 => VarClass<float32, (4573,)>
5 => VarClass<float32, (4573,)>
6 => VarClass<float32, (4573,)>
7 => VarClass<float32, (4573,)>
8 => VarClass<int32, (9874, 2)>
9 => VarClass<int32, (4573, 5)>
10 => VarClass<int32, (9874,)>
11 => VarClass<int32, (9874,)>
12 => VarClass<float32, (4573, 3)>
13 => VarClass<float32, (9874, 3)>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


284 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
# verify

def verify(a, decimal, plot):
  b = VarClass.to_device(a)
  if plot == True:
    plt.plot(a, label="cpu")
    plt.plot(b, label="gpu")
    plt.legend()
    plt.show()
  np.testing.assert_almost_equal(a, b, decimal=decimal)


for i, arg in enumerate(args):
  print(f"{i + 1} => test arg {arg}")
  verify(arg, decimal=2, plot=False)


1 => test arg [1. 1. 1. ... 1. 1. 1.]
2 => test arg [0. 0. 0. ... 0. 0. 0.]
3 => test arg []
4 => test arg [0. 0. 0. ... 0. 0. 0.]
5 => test arg [0. 0. 0. ... 0. 0. 0.]
6 => test arg [0. 0. 0. ... 0. 0. 0.]
7 => test arg [1. 1. 1. ... 1. 1. 1.]
8 => test arg [[4197   -1]
 [4243   -1]
 [4215   -1]
 ...
 [1361 2321]
 [1342 1361]
 [2321 3600]]
9 => test arg [[2074 3314 3312 3316    4]
 [4109 5081 5072 5083    4]
 [2182 5571 5573 5574    4]
 ...
 [2608 1495 2596 2609    4]
 [1350 1517 1518 1513    4]
 [1350 1348  224 1351    4]]
10 => test arg [1 1 1 ... 0 0 0]
11 => test arg [0 0 0 ... 0 0 0]
12 => test arg [[0.44276604 0.43170595 0.5221834 ]
 [0.7694712  0.24118435 0.90514314]
 [0.7770873  0.31323838 0.8029697 ]
 ...
 [0.92618835 0.5116935  0.9457144 ]
 [0.03945985 0.9010808  0.98169875]
 [0.02115859 0.8817337  0.96096134]]
13 => test arg [[0.         0.02886751 0.45000002]
 [0.         0.54647523 0.03078931]
 [0.         0.97113246 0.55      ]
 ...
 [0.72506887 0.71881676 0.87177306]
 [